In [ ]:
import requests
from bs4 import BeautifulSoup
import pyautogui
import pyperclip
import pandas as pd
import webbrowser
import time
import re

# ==============================================================================
# ⚙️ CONFIGURAÇÕES
# ==============================================================================

# Lista exata de páginas que o robô vai ler (Ordene pelas páginas com os trabalhos do seu interesse)
URLS_ALVO = [
    "https://www.99freelas.com.br/projects?categoria=design-e-criacao&page=1",
    #"https://www.99freelas.com.br/projects?categoria=design-e-criacao&page=2",
    #"https://www.99freelas.com.br/projects?categoria=fotografia-e-audiovisual&page=1",
    #"https://www.99freelas.com.br/projects?categoria=fotografia-e-audiovisual&page=2",
    #"https://www.99freelas.com.br/projects?categoria=vendas-e-marketing&page=1",
    #"https://www.99freelas.com.br/projects?categoria=vendas-e-marketing&page=2",
    #"https://www.99freelas.com.br/projects?categoria=escrita&page=1",
    #"https://www.99freelas.com.br/projects?categoria=escrita&page=2",
    #"https://www.99freelas.com.br/projects?categoria=web-mobile-e-software&page=1",
    #"https://www.99freelas.com.br/projects?categoria=web-mobile-e-software&page=2"
]

TEXTO_PROPOSTA = """Olá! Analisei sua demanda e tenho a expertise necessária para entregá-la com alto padrão de qualidade.
Prezo pela agilidade, comunicação clara e cumprimento rigoroso dos prazos.
A proposta abaixo visa o melhor custo-benefício para o seu projeto.
Um preço de ENTRADA, pois estou buscando meus primeiros projetos para subir minha reputação na plataforma!
Estou à disposição para iniciar de imediato! Aguardando seu retorno.

Portfólio: https://linktr.ee/shelter.hub
"""

# Segurança
pyautogui.FAILSAFE = True 

# ==============================================================================
# 🧠 FUNÇÕES DE INTELIGÊNCIA
# ==============================================================================

def limpar_valor_monetario(texto):
    if not texto or texto == "Não identificado": return 0.0
    apenas_numeros = re.sub(r'[^\d,]', '', texto).replace(',', '.')
    try: return float(apenas_numeros)
    except: return 0.0

def limpar_dias(texto):
    if not texto or texto == "Não identificado": return 0
    match = re.search(r'\d+', texto)
    if match: return int(match.group())
    return 0

def calcular_lance_estrategico(media_valor_str, media_prazo_str):
    print(f"   [Cálculo] Base: Valor='{media_valor_str}' | Prazo='{media_prazo_str}'")
    
    val_float = limpar_valor_monetario(media_valor_str)
    dias_int = limpar_dias(media_prazo_str)
    
    novo_valor = val_float - 1
    novo_prazo = dias_int - 1
    
    if novo_valor =< 50: novo_valor = 97 
    if novo_prazo =< 1: novo_prazo = 2
    
    # Formatação corrigida para input mask (Vírgula e decimais)
    str_valor = f"{novo_valor:.2f}".replace('.', ',')
    str_prazo = str(novo_prazo)
    
    return str_valor, str_prazo

def extrair_dados_do_texto(texto):
    info = {'Cliente': "Não identificado", 'Orçamento': "Não identificado", 
            'Média Bids': "Não identificado", 'Duração Média': "Não identificado"}
    
    linhas = texto.split('\n')
    for i, linha in enumerate(linhas):
        l = linha.strip()
        if "Cliente:" in l or "Publicado por" in l:
            info['Cliente'] = l.replace("Cliente:", "").strip()
        if "Orçamento" in l:
            info['Orçamento'] = l
        if "Média das propostas" in l or "Valor médio" in l:
            conteudo = l
            if i+1 < len(linhas): conteudo += " " + linhas[i+1]
            info['Média Bids'] = conteudo
        if "Duração média" in l or "Prazo médio" in l:
            conteudo = l
            if i+1 < len(linhas): conteudo += " " + linhas[i+1]
            info['Duração Média'] = conteudo
    return info

# ==============================================================================
# 🕵️ COLETA DE LINKS (BS4)
# ==============================================================================
def coletar_links_bs4(url):
    print(f"Lendo página: {url}")
    headers = {'User-Agent': "Mozilla/5.0"}
    try:
        r = requests.get(url, headers=headers)
        soup = BeautifulSoup(r.text, 'html.parser')
        links = []
        for h1 in soup.find_all('h1', class_='title'):
            a = h1.find('a')
            if a:
                full_link = "https://www.99freelas.com.br" + a['href']
                links.append(full_link)
        return links
    except Exception as e:
        print(f"Erro na url {url}: {e}")
        return []

# ==============================================================================
# 🤖 AUTOMAÇÃO VISUAL
# ==============================================================================
def ler_tela():
    time.sleep(1)
    pyautogui.click(x=960, y=540) # Ajuste X,Y para o meio da sua tela
    pyautogui.hotkey('ctrl', 'a')
    time.sleep(0.5)
    pyautogui.hotkey('ctrl', 'c')
    time.sleep(0.5)
    return pyperclip.paste()

def preencher_proposta(valor, prazo):
    print(f"   -> Preenchendo formulário: R$ {valor} | {prazo} dias")
    
    # 1. Rolar para o fundo
    pyautogui.scroll(-2000)
    time.sleep(1)
    
    # 2. Achar campo de texto usando Localizar do navegador
    pyautogui.hotkey('ctrl', 'f')
    time.sleep(0.5)
    pyautogui.write("oferta") # Texto âncora
    time.sleep(0.5)
    pyautogui.press('esc')
    time.sleep(0.5)
    
    # 3. Navegar até a caixa de texto (O PULO DO GATO É AQUI)
    # Se falhar, aumente ou diminua o número de tabs
    pyautogui.press('tab')
    pyautogui.press('tab') 
    time.sleep(0.2)
    
    # Vai para Valor
    time.sleep(0.2)
    pyautogui.write(valor)

    # Vai para Prazo
    pyautogui.press('tab')
    pyautogui.press('tab')
    time.sleep(0.2)
    pyautogui.write(prazo)

    # Escreve descrição
    pyautogui.press('tab')
    pyperclip.copy(TEXTO_PROPOSTA)
    pyautogui.hotkey('ctrl', 'v')
    time.sleep(0.5)
    
    
    # ⚠️ IMPORTANTE: O Enter final está comentado para segurança
    # Quando estiver confiante, descomente as duas linhas abaixo:
    
    pyautogui.press('tab') # Foca botão enviar
    pyautogui.press('enter') # Envia
    time.sleep(3)
    
    print("   -> Proposta preenchida (Envio simulado).")

# ==============================================================================
# 🚀 MAIN
# ==============================================================================
def main():
    todos_links = []
    
    # 1. Varredura inicial (Rápida)
    print(f"--- Iniciando coleta de {len(URLS_ALVO)} categorias ---")
    for url_cat in URLS_ALVO:
        links_pagina = coletar_links_bs4(url_cat)
        todos_links.extend(links_pagina)
        time.sleep(1) # Pausa leve
        
    print(f"\nTotal encontrado: {len(todos_links)} projetos.")
    print("--- INICIANDO ROBÔ VISUAL EM 5 SEGUNDOS ---")
    print("Solte o mouse!")
    time.sleep(5)
    
    dados_relatorio = []
    
    # 2. Processamento Visual (Um por um)
    for i, link_proj in enumerate(todos_links):
        print(f"\n[{i+1}/{len(todos_links)}] Processando...")
        
        try:
            # Abre Projeto
            webbrowser.open(link_proj)
            time.sleep(6)
            info_proj = extrair_dados_do_texto(ler_tela())
            
            # Fecha e Abre Bid
            pyautogui.hotkey('ctrl', 'w')
            time.sleep(1)
            
            link_bid = link_proj.replace("/project/", "/project/bid/")
            webbrowser.open(link_bid)
            time.sleep(6)
            
            # Captura Médias e Calcula
            info_bid = extrair_dados_do_texto(ler_tela())
            
            media_val = info_bid['Média Bids'] if info_bid['Média Bids'] != "Não identificado" else "100"
            media_prz = info_bid['Duração Média'] if info_bid['Duração Média'] != "Não identificado" else "7"
            
            lance_val, lance_prz = calcular_lance_estrategico(media_val, media_prz)
            
            # Preenche
            preencher_proposta(lance_val, lance_prz)
            
            # Salva
            dados_relatorio.append({
                "Link": link_proj,
                "Orçamento Original": info_proj['Orçamento'],
                "Média Mercado": media_val,
                "Meu Lance": lance_val,
                "Status": "Preenchido"
            })
            
            # Fecha aba final
            pyautogui.hotkey('ctrl', 'w')
            time.sleep(2)
            
        except Exception as e:
            print(f"Erro: {e}")
            pyautogui.hotkey('ctrl', 'w') # Tenta fechar para não travar
            
    # Exporta
    if dados_relatorio:
        pd.DataFrame(dados_relatorio).to_excel("Relatorio_Geral_Categorias.xlsx", index=False)
        print("Sucesso! Planilha gerada.")

if __name__ == "__main__":
    main()

--- Iniciando coleta de 10 categorias ---
Lendo página: https://www.99freelas.com.br/projects?categoria=design-e-criacao&page=1
Lendo página: https://www.99freelas.com.br/projects?categoria=design-e-criacao&page=2
Lendo página: https://www.99freelas.com.br/projects?categoria=fotografia-e-audiovisual&page=1
Lendo página: https://www.99freelas.com.br/projects?categoria=fotografia-e-audiovisual&page=2
Lendo página: https://www.99freelas.com.br/projects?categoria=vendas-e-marketing&page=1
Lendo página: https://www.99freelas.com.br/projects?categoria=vendas-e-marketing&page=2
Lendo página: https://www.99freelas.com.br/projects?categoria=escrita&page=1
Lendo página: https://www.99freelas.com.br/projects?categoria=escrita&page=2
Lendo página: https://www.99freelas.com.br/projects?categoria=web-mobile-e-software&page=1
Lendo página: https://www.99freelas.com.br/projects?categoria=web-mobile-e-software&page=2

Total encontrado: 100 projetos.
--- INICIANDO ROBÔ VISUAL EM 5 SEGUNDOS ---
Solte o m

FailSafeException: PyAutoGUI fail-safe triggered from mouse moving to a corner of the screen. To disable this fail-safe, set pyautogui.FAILSAFE to False. DISABLING FAIL-SAFE IS NOT RECOMMENDED.